In [ ]:
!pip install flair

In [ ]:
from flair.data import Sentence
from flair.embeddings import TransformerDocumentEmbeddings
import numpy as np
import pandas as pd
import torch

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
filenames = ["ArticlesMay2017.csv"]

In [5]:
def read_article_from_dataset(filename: str) -> pd.DataFrame:
    return pd.read_csv(filename)[["articleID", "headline", "keywords"]]

In [6]:
article_sets = [(filename, read_article_from_dataset("gdrive/MyDrive/Research/NewYorkTimesComments/" + filename)) for filename in filenames]

In [7]:
embedding_model = TransformerDocumentEmbeddings('bert-large-uncased', fine_tune = False)

Some weights of the model checkpoint at bert-large-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [8]:
from tqdm import tqdm

def compute_embeddings(articles):
  embeddings = []

  for article in tqdm(articles.itertuples()):
        article_sentence = Sentence(article.keywords)
        embedding = embedding_model.embed(article_sentence)
        embeddings.append({"ID": article.articleID,
                          "headline": article.headline,
                          "keywords": article.keywords,
                          "embedding": article_sentence.embedding})
  return embeddings

In [10]:
for article in article_sets:
  embeddings = compute_embeddings(article[1])
  torch.save(embeddings, "gdrive/MyDrive/Research/embedded_article_" + article[0].split(".")[0] + "_bert-large-uncased.pt")

996it [00:20, 49.67it/s]
